In [1]:
# import dependencies
import random
import numpy as np
import pandas as pd
import timeit
import matplotlib.pyplot as plt
from citipy import citipy
from nturl2path import pathname2url
import pprint as pprint
import requests
from config import ow_key
import time
from datetime import datetime
from scipy.stats import linregress

pp = pprint.PrettyPrinter(indent = 4, width =80, compact = False)

In [2]:
# make 1500 random coordinate pairs
coords = list(zip(np.random.uniform(-90,90,1500),np.random.uniform(-90,90,1500)))

# use citipy to find and make list of the nearest cities to the random coordinates
cities = []
for i in coords:
    city = citipy.nearest_city(i[0],i[1]).city_name
    if city not in cities:
        cities.append(city)

In [3]:
coords

[(-77.09852825369714, -77.96446785549745),
 (88.01120234439591, -55.73576858709168),
 (-73.51850674833261, -18.932906143494662),
 (13.046972690070362, -49.4064814674502),
 (87.30026105461849, -55.76490658028853),
 (87.95083171183046, 56.393963653426994),
 (2.3418467684055884, -12.053137785583047),
 (65.04929576135231, 81.30403039538888),
 (-72.52900100102066, -44.614198713398004),
 (62.59611796539019, -18.62935313386363),
 (1.3278105275989844, 43.95514832281688),
 (-55.75747162144376, 89.68265878920357),
 (26.884826586422378, -88.03040412806615),
 (-21.71197583949072, 41.58986063652878),
 (67.1370661548626, 18.017384369051598),
 (43.505703727760874, -58.1312763767677),
 (-18.217237028605098, 72.66767888206573),
 (56.66995700774606, -12.946203877829817),
 (66.13829091383235, 39.37431523198288),
 (-36.0253683230922, -17.21180576234225),
 (-80.92959650578045, -17.35525808757704),
 (-59.36752993659312, 37.441069432312574),
 (-47.11718122318338, 41.743581912625075),
 (3.5019046816525616, 85

In [ ]:
#check to see there are at least 500 unique cities in list
print('There are', len(cities),'cities. No duplicates?',len(cities) == len(set(cities)))

In [ ]:
url = 'https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=' + ow_key

In [ ]:
#create empty lis to hold weather data

city_data = []

# print start of log

print('Beginning Data Retrieval        ')
print('--------------------------------')

#make counters
record_count = 1
set_count = 1

#loop through cities in list 
for i, city in enumerate(cities):

    #50 at a time due to free tier limits
    if(i % 50==0 and i >=50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    #create endpoint url for each city
    city_url = url + '&q=' + city.replace(' ','+')

    # log the url, record, amd set numbers and the city
    print( f'Processing Record {record_count} of Set {set_count} | {city}')

    # add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# convert data to pandas dataframe
df = pd.DataFrame(city_data)
df.head()

In [ ]:
# reorder columns
df = df[['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']]
df.head()

In [ ]:
# create output csv
file = '../UNCBC-HW6-PlanMyTripApp/weather_data/cities.csv'
df.to_csv(file,index_label = 'City_ID')

In [ ]:
# get date 
today = time.strftime('%x')

#build scatter plot, Lat v Max Temp
plt.scatter(df['Lat'],
            df['Max Temp'],
            edgecolor='k',
            linewidths=1,
            marker='o',
            alpha=0.8,
            label='Cities')

plt.title(f'City Latitude vs Max Temperature ' + time.strftime('%x'))
plt.ylabel('Max Temperature (F)')
plt.xlabel('Latitude')
plt.grid(True)


#save plot
plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig1.png')

#show plot
plt.show()

In [ ]:
#build scatter plot, Lat v Humidity
plt.scatter(df['Lat'],
            df['Humidity'],
            edgecolor='k',
            linewidths=1,
            marker='o',
            alpha=0.8,
            label='Cities')

plt.title(f'City Latitude vs Humidity ' + time.strftime('%x'))
plt.ylabel('Humidity (%)')
plt.xlabel('Latitude')
plt.grid(True)


#save plot
plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig2.png')

#show plot
plt.show()

In [ ]:
#build scatter plot, Lat v Cloudiness
plt.scatter(df['Lat'],
            df['Cloudiness'],
            edgecolor='k',
            linewidths=1,
            marker='o',
            alpha=0.8,
            label='Cities')

plt.title(f'City Latitude vs Cloudiness ' + time.strftime('%x'))
plt.ylabel('Cloudiness (%)')
plt.xlabel('Latitude')
plt.grid(True)


#save plot
plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig3.png')

#show plot
plt.show()

In [ ]:
#build scatter plot, Lat v Wind Speed
plt.scatter(df['Lat'],
            df['Wind Speed'],
            edgecolor='k',
            linewidths=1,
            marker='o',
            alpha=0.8,
            label='Cities')

plt.title(f'City Latitude vs Wind Speed ' + time.strftime('%x'))
plt.ylabel('Wind Speed (mph)')
plt.xlabel('Latitude')
plt.grid(True)


#save plot
plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig4.png')

#show plot
plt.show()

In [ ]:
# lin regression practice

# Create an equal number of latitudes and temperatures.
lats = [42.5, 43.9, 8.1, 36.8, 79.9, 69.1, 25.7, 15.3, 12.7, 64.5]
temps = [80.5, 75.3, 90.9, 90.0, 40.4, 62.3, 85.4, 79.6, 72.5, 72.0]

# Perform linear regression.
(slope, intercept, r_value, p_value, std_err) = linregress(lats, temps)
# Get the equation of the line.
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
print(line_eq)
print(f"The p-value is: {p_value:.3f}")

#another way to access attributes of linregress
# round(linregress(lats,temps).slope,2)

# Calculate the regression line "y values" from the slope and intercept.
regress_values = [(lat * slope + intercept) for lat in lats]

# Create a scatter plot of the x and y values.
plt.scatter(lats,temps)

# Plot the regression line with the x-values and the y coordinates based on the intercept and slope.
plt.plot(lats,regress_values,"r")

# Annotate the text for the line equation and add its coordinates.
plt.annotate(line_eq, (10,40), fontsize=15, color="red")
plt.xlabel('Latitude')
plt.ylabel('Temp')

#show the plot

#save the plot
#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')
plt.show()

In [ ]:
# make function to perform and plot linear regression model

def lm(x,y,title,subtitle,ylab,xlab,text_coords):
        # Run regression on hemisphere weather data.
    (slope, intercept, r_value, p_value, std_err) = linregress(x, y)
    print('r=',r_value)

    # Calculate the regression line "y values" from the slope and intercept.
    regress_values = x * slope + intercept
    # Get the equation of the line.
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    # Create a scatter plot and plot the regression line.
    plt.scatter(x,y)
    plt.plot(x,regress_values,"r")
    # Annotate the text for the line equation.
    plt.annotate(line_eq, text_coords, fontsize=15, color="red")
    plt.suptitle(title)
    plt.title(subtitle)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show()


In [ ]:
# make df with cities in northern hemisphere (lat >=0)
n_df = df[df['Lat']>=0]
n_df

In [ ]:
# make df with cities in southern hemisphere (lat >=0)
s_df = df[df['Lat']<0]
s_df

In [ ]:
# Northern Hemi Plot, Lat v Max Temp 
lm(x=n_df['Lat'],y=n_df['Max Temp'],
xlab='Latitude',ylab='Maximum Temperature',
title='Linear Regression for Maximum Temperature',
subtitle = 'Northern Hemisphere',
text_coords = (46,8))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')

In [ ]:
# Southern Hemi Plot, Lat v Max Temp 
lm(x=s_df['Lat'],y=s_df['Max Temp'],
xlab='Latitude',ylab='Maximum Temperature',
title='Linear Regression for Maximum Temperature',
subtitle = 'Southern Hemisphere',
text_coords = (-22,42))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')

In [ ]:
# Northern Hemi Plot, Lat v Humidity
lm(x=n_df['Lat'],y=n_df['Humidity'],
xlab='Latitude',ylab='Humidity (%)',
title='Linear Regression for Percent Humidity',
subtitle = 'Northern Hemisphere',
text_coords = (46,8))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')

In [ ]:
# Southern Hemi Plot, Lat v Humidity
lm(x=s_df['Lat'],y=s_df['Humidity'],
xlab='Latitude',ylab='Humidity (%)',
title='Linear Regression for Percent Humidity',
subtitle = 'Southern Hemisphere',
text_coords = (-20,10))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')

In [ ]:
# Northern Hemi Plot, Lat v Cloudiness
lm(x=n_df['Lat'],y=n_df['Cloudiness'],
xlab='Latitude',ylab='Cloudiness (%)',
title='Linear Regression for Percent Cloudiness',
subtitle = 'Northern Hemisphere',
text_coords = (48,0))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')

In [ ]:
# Southern Hemi Plot, Lat v Cloudiness
lm(x=s_df['Lat'],y=s_df['Cloudiness'],
xlab='Latitude',ylab='Cloudiness (%)',
title='Linear Regression for Percent Cloudiness',
subtitle = 'Southern Hemisphere',
text_coords = (-20,0))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')

In [ ]:
# Northern Hemi Plot, Lat v Wind Speed
lm(x=n_df['Lat'],y=n_df['Wind Speed'],
xlab='Latitude',ylab='Wind Speed (mph)',
title='Linear Regression for Wind Speed',
subtitle = 'Northern Hemisphere',
text_coords = (48,0))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')

In [ ]:
# Douthern Hemi Plot, Lat v Wind Speed
lm(x=s_df['Lat'],y=s_df['Wind Speed'],
xlab='Latitude',ylab='Wind Speed (mph)',
title='Linear Regression for Wind Speed',
subtitle = 'Southern Hemisphere',
text_coords = (-22,0))

#plt.savefig('../UNCBC-HW6-PlanMyTripApp/weather_data/Fig0.png')